In [10]:

import pandas as pd
import glob

archivos = {
    2020: '../data/raw/acc_2020.xlsx', 
    2021: '../data/raw/acc_2021.xlsx', 
    2022: '../data/raw/acc_2022.xlsx', 
    2023: '../data/raw/acc_2023.xlsx',
    2024: '../data/raw/acc_2024.xlsx'
}

lista_df = []

for anyo, ruta in archivos.items():
    print(f"Cargando año {anyo}...")
    df_temp = pd.read_excel(ruta)
    # Es vital que todos tengan las mismas columnas
    lista_df.append(df_temp)

# Unimos todos en uno solo
df_historico = pd.concat(lista_df, ignore_index=True)

print(f"✅ Total de accidentes cargados: {len(df_historico)}")


Cargando año 2020...
Cargando año 2021...
Cargando año 2022...
Cargando año 2023...
Cargando año 2024...
✅ Total de accidentes cargados: 464039


In [14]:
# 1. Identificamos las columnas que causan problemas (las tipo 'object' o texto)
# Vamos a convertir todas las columnas de texto a string puro para limpiar mezclas de int/str
cols_texto = df_historico.select_dtypes(include=['object']).columns

for col in cols_texto:
    # Convertimos a string y reemplazamos los nulos por una cadena vacía o "DESCONOCIDO"
    df_historico[col] = df_historico[col].astype(str).fillna("DESCONOCIDO")

# 2. Limpiamos nombres de columnas (por si acaso)
df_historico.columns = df_historico.columns.str.strip().str.upper()

# 3. Ahora intenta guardar de nuevo
df_historico.to_parquet('../data/processed/accidentes_2020_2024.parquet', engine='pyarrow', index=False)

print("✅ ¡Ahora sí! Parquet guardado sin errores de tipo.")

✅ ¡Ahora sí! Parquet guardado sin errores de tipo.


In [27]:
import pandas as pd
import joblib
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder

# 1. Cargar datos
df = pd.read_parquet('../data/processed/accidentes_2020_2024.parquet')

# 2. Crear Target y Variables (Feature Engineering Directo)
df['ES_MORTAL'] = (df['TOTAL_MU30DF'] > 0).astype(int)

# IMPORTANTE: Convertimos todo a string y calculamos ANTES del pipeline
df['DIA_SEMANA'] = df['DIA_SEMANA'].astype(str).str.upper()
df['COND_METEO'] = df['CONDICION_METEO'].astype(str).str.upper()
df['COND_LUZ'] = df['CONDICION_ILUMINACION'].astype(str).str.upper()

# Variables extra
df['ES_FIN_SEMANA'] = df['DIA_SEMANA'].isin(['SÁBADO', 'DOMINGO', 'SABADO', '6', '7']).astype(int)
df['ES_NOCTURNO'] = ((df['HORA'] >= 22) | (df['HORA'] <= 6)).astype(int)
df['RIESGO_VISIBILIDAD'] = ((df['COND_METEO'] != 'DESPEJADO') & (df['COND_LUZ'] != 'PLENO DÍA')).astype(int)

# Seleccionamos las columnas finales
features = [
    'HORA', 'MES', 'DIA_SEMANA', 'COD_PROVINCIA', 'TIPO_VIA', 
    'CONDICION_METEO', 'CONDICION_ILUMINACION', 'ZONA_AGRUPADA',
    'ES_FIN_SEMANA', 'ES_NOCTURNO', 'RIESGO_VISIBILIDAD'
]

X = df[features]
y = df['ES_MORTAL']

# 3. Preprocesador (Solo para las columnas de texto)
cat_features = ['DIA_SEMANA', 'COD_PROVINCIA', 'TIPO_VIA', 'CONDICION_METEO', 'CONDICION_ILUMINACION', 'ZONA_AGRUPADA']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), cat_features)
    ],
    remainder='passthrough'
)

# 4. Pipeline
modelo_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=100, # Reducido para que no se cuelgue tu PC
        max_depth=6,
        learning_rate=0.1,
        scale_pos_weight=100,
        random_state=42
    ))
])

# 5. Entrenar y Guardar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("🧠 Entrenando... (esto puede tardar un minuto)")
modelo_pipeline.fit(X_train, y_train)

joblib.dump(modelo_pipeline, '../data/processed/modelo_vial_v3.pkl')
print("✅ MODELO GUARDADO CORRECTAMENTE")

🧠 Entrenando... (esto puede tardar un minuto)
✅ MODELO GUARDADO CORRECTAMENTE


In [25]:
from sklearn.metrics import classification_report, confusion_matrix

# 1. Realizar predicciones con los datos de prueba
y_pred = modelo_pipeline.predict(X_test)

# 2. Imprimir el reporte detallado
print("\n" + "="*40)
print("📊 ESTADÍSTICAS DEL MODELO V3 (ENRIQUECIDO)")
print("="*40)
print(classification_report(y_test, y_pred))

# 3. Ver la importancia de las nuevas variables
importances = modelo_pipeline.named_steps['classifier'].feature_importances_
# Sacamos los nombres de las columnas después del preprocesador
feature_names = cat_features + ['HORA', 'MES', 'ES_FIN_SEMANA', 'ES_NOCTURNO', 'RIESGO_VISIBILIDAD']
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
print("\n🔍 IMPORTANCIA DE LAS VARIABLES:")
print(feature_importance_df.sort_values(by='importance', ascending=False).head(5))
print("="*40)


📊 ESTADÍSTICAS DEL MODELO V3 (ENRIQUECIDO)
              precision    recall  f1-score   support

           0       0.99      0.65      0.79     91274
           1       0.03      0.66      0.06      1534

    accuracy                           0.65     92808
   macro avg       0.51      0.65      0.42     92808
weighted avg       0.98      0.65      0.77     92808


🔍 IMPORTANCIA DE LAS VARIABLES:
                 feature  importance
5          ZONA_AGRUPADA    0.478637
9            ES_NOCTURNO    0.102108
1          COD_PROVINCIA    0.070327
4  CONDICION_ILUMINACION    0.067421
2               TIPO_VIA    0.065929


In [28]:
import joblib

# Carga el modelo
pipeline = joblib.load('../data/processed/modelo_vial_v3.pkl')

# Extraer los nombres de las columnas que el modelo "vio" al entrenar
if hasattr(pipeline, "feature_names_in_"):
    cols = pipeline.feature_names_in_
    print("📋 El modelo espera estas columnas (y en este orden):")
    print(list(cols))
else:
    # Si no están en el pipeline, están en el primer paso (el preprocessor)
    cols = pipeline.named_steps['preprocessor'].feature_names_in_
    print("📋 El modelo espera estas columnas:")
    print(list(cols))

📋 El modelo espera estas columnas (y en este orden):
['HORA', 'MES', 'DIA_SEMANA', 'COD_PROVINCIA', 'TIPO_VIA', 'CONDICION_METEO', 'CONDICION_ILUMINACION', 'ZONA_AGRUPADA', 'ES_FIN_SEMANA', 'ES_NOCTURNO', 'RIESGO_VISIBILIDAD']


In [29]:
# Acceder al encoder dentro del preprocesador
preprocessor = pipeline.named_steps['preprocessor']
encoder = preprocessor.named_transformers_['cat']

# Ver las categorías de cada columna categórica
for col, cats in zip(cat_features, encoder.categories_):
    print(f"\n🔍 Valores válidos para '{col}':")
    print(cats[:10], "... (y más)") # Mostramos los primeros 10


🔍 Valores válidos para 'DIA_SEMANA':
['1' '2' '3' '4' '5' '6' '7'] ... (y más)

🔍 Valores válidos para 'COD_PROVINCIA':
['1' '10' '11' '12' '13' '14' '15' '16' '17' '18'] ... (y más)

🔍 Valores válidos para 'TIPO_VIA':
['1' '10' '11' '12' '13' '14' '2' '3' '4' '5'] ... (y más)

🔍 Valores válidos para 'CONDICION_METEO':
['1' '2' '3' '4' '5' '6' '7' '999'] ... (y más)

🔍 Valores válidos para 'CONDICION_ILUMINACION':
['1' '2' '3' '4' '5' '6' '999'] ... (y más)

🔍 Valores válidos para 'ZONA_AGRUPADA':
['1' '2'] ... (y más)
